In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os

import spacy, pickle

import torch

from torchtext import data
from torchtext import datasets

import random
import inspect

In [3]:
path = "./"
save_data_path = path + 'save_data/'
large_save_data_path = '/notebooks/large-storage/'
print(os.listdir(path))

['test.tsv', 'prepare-word-embedding-nlp.ipynb', 'tokenization.ipynb', 'test-batching-padding.ipynb', 'train.tsv', 'sampleSubmission.csv', 'save_data', '.ipynb_checkpoints']


In [4]:
loaded_data = pickle.load(open(save_data_path + 'pre-processed-data.pkl', 'rb'))
loaded_kaggle_test = pickle.load(open(save_data_path + 'pre-processed-kaggle-test.pkl', 'rb'))
loaded_vocab = pickle.load(open(save_data_path + 'genereated-vocab.pkl', 'rb'))

In [5]:
loaded_data.head()

,PhraseId,SentenceId,Phrase,Sentiment,Phrase_length,Tokenized_phrase,Indexed_phrase
0,1,1,A series of escapades demonstrating the adage ...,1,188,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
1,2,1,A series of escapades demonstrating the adage ...,2,77,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
2,3,1,A series,2,8,"[xxbos, a, series, xxeos]","[2, 10, 341, 3]"
3,4,1,A,2,1,"[xxbos, a, xxeos]","[2, 10, 3]"
4,5,1,series,2,6,"[xxbos, series, xxeos]","[2, 341, 3]"


## Testing

In [6]:
type(loaded_data)

pandas.core.frame.DataFrame

In [11]:
processed_nlp = pickle.load(open(large_save_data_path + 'processed-nlp.pkl', 'rb'))

MemoryError: 

In [7]:
PHRASE_ID = data.Field(use_vocab = False)
TEXT = data.Field(include_lengths = True, use_vocab = False)
LABEL = data.LabelField()

In [8]:
fields = [('id', PHRASE_ID), ('text', TEXT), ('label', LABEL)]

In [9]:
len(loaded_data['Phrase'])

156060

In [10]:
examples = []
length = len(loaded_data['Phrase'])
for i in range(length):
    examples.append(data.Example.fromlist([ [loaded_data['PhraseId'][i]], loaded_data['Indexed_phrase'][i], loaded_data['Sentiment'][i]], fields))
    
examples[:10]

In [11]:
data_set = data.Dataset(examples, fields)

In [12]:
data_set.sort_key = lambda x: len(x.text)

In [13]:
SEED = 1234

In [14]:
train_data, valid_data, test_data = data_set.split([0.8, 0.1, 0.1], random_state = random.seed(SEED))

In [15]:
train_data.sort_key

<function __main__.<lambda>(x)>

In [16]:
len(data_set)*0.1

15606.0

In [17]:
len(train_data)

124848

In [18]:
len(valid_data)

15606

In [19]:
len(test_data)

15606

In [20]:
train_data[10].text

[2, 51, 10, 279, 1617, 19, 10995, 1188, 184, 59, 70, 23, 196, 683, 3]

In [21]:
train_data[10].label

1

In [22]:
train_data[10].id

AttributeError: 'Example' object has no attribute 'id'

In [22]:
loaded_data['Indexed_phrase'][4569-1]

[2, 51, 10, 279, 1617, 19, 10995, 1188, 184, 59, 70, 23, 196, 683, 3]

In [23]:
loaded_data['Sentiment'][4569-1]

1

In [24]:
loaded_data['Phrase'][4569-1]

'has a visual flair that waxes poetic far too much for our taste'

In [25]:
inspect.getsource(train_data.sort_key)

'data_set.sort_key = lambda x: len(x.text)\n'

In [26]:
# PHRASE_ID.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [27]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [36]:
raw_train_iterator = iter(train_iterator)
batch = next(raw_train_iterator)

array
[[10572], [87216], [52954], [149131], [71655], [112640], [38001], [126955], [143535], [140021], [95693], [151182], [125778], [14090], [42906], [96546], [68666], [32416], [71289], [106765], [92049], [83315], [86966], [120060], [127704], [45468], [69025], [82903], [21256], [23322], [63642], [154946], [33347], [149147], [69347], [1841], [43556], [62670], [105789], [70454], [22234], [51192], [39641], [64422], [94377], [130998], [63155], [44136], [126718], [87609], [137493], [19468], [5769], [144038], [23798], [90046], [148776], [132418], [144202], [152531], [29978], [65092], [34700], [120382]]
type
torch.int64
cpu
array
[[2, 5604, 8, 3544, 42, 46, 11027, 191, 3], [2, 4176, 1211, 641, 11, 1925, 12, 888, 3], [2, 10, 702, 9, 8355, 415, 11, 64, 3], [2, 18, 655, 45, 7, 1791, 16, 921, 3], [2, 430, 115, 24, 517, 264, 337, 15, 3], [2, 18, 17, 308, 11, 10, 11634, 5554, 3], [2, 7, 20, 130, 41, 31, 520, 4261, 3], [2, 10, 25, 9, 12, 161, 10, 8693, 3], [2, 10, 1648, 9378, 6229, 13, 6230, 249, 3],

TypeError: an integer is required (got type str)

In [29]:
a, b = batch.text

In [30]:
b

tensor([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13])

In [31]:
a[:, 60]

tensor([   2,  123,   80,    7, 3484,    7, 4344,  266, 3960,   12,  748,   15,
           3])

In [32]:
batch.id[:, 60]

tensor([111789])

In [33]:
loaded_data['Indexed_phrase'][111789-1]

[2, 123, 80, 7, 3484, 7, 4344, 266, 3960, 12, 748, 15, 3]

In [34]:
len(loaded_data['Indexed_phrase'][109457-1])

13

In [35]:
type(loaded_data['Indexed_phrase'][0][0])

int

In [45]:
for i in range(len(loaded_data['Indexed_phrase'])):
    if type(loaded_data['Indexed_phrase'][i]) is str:
        print(loaded_data['Indexed_phrase'][i])
        print(i)
        break
    for j in range(len(loaded_data['Indexed_phrase'][i])):
        if type(loaded_data['Indexed_phrase'][i][j]) is str:
            print(i)
            print(loaded_data['Indexed_phrase'][i][j])
            break